# Imports

In [ ]:
import requests
import time
import json
import math
import pandas as pd
import numpy as np
import pymongo 
from pymongo import MongoClient
from api_key import x

# Defining classes and initializing MongoDB

In [ ]:
class RiotAPI:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "https://na1.api.riotgames.com/lol"
        self.headers = {'X-Riot-Token': self.api_key}

    def perform_request(self, api_url):
        while True:
            response = requests.get(api_url, headers=self.headers)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:
                print("Rate limit exceeded, waiting 10 seconds to retry...")
                time.sleep(10)
            else:
                print(f"HTTP request failed with status {response.status_code}. No data fetched.")
                return None

    def get_puuid(self, summoner_id):
        """ Fetch the PUUID for a given Summoner ID """
        api_url = f"{self.base_url}/summoner/v4/summoners/{summoner_id}"
        summoner_info = self.perform_request(api_url)
        if summoner_info:
            return summoner_info.get('puuid')
        print(f"No PUUID found for Summoner ID {summoner_id}.")
        return None

class SummonerDataCollector:
    def __init__(self, api_key, mongodb_uri):
        self.api = RiotAPI(api_key)
        self.client = MongoClient(mongodb_uri)
        self.db = self.client['leagueData']
        self.collection = self.db['summoner_data']

    def gather_summoner_data(self, tier, division, start_page=1, num_pages=1):
        summoner_data = []
        for page in range(start_page, start_page + num_pages):
            api_url = f"{self.api.base_url}/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
            data = self.api.perform_request(api_url)
            if data:
                for entry in data:
                    summoner_data.append({
                        'summonerId': entry.get('summonerId'),
                        'tier': tier,
                        'division': division
                    })
        self.collection.insert_many(summoner_data)

class PUUIDConverter:
    def __init__(self, api_key, mongodb_uri):
        self.api = RiotAPI(api_key)
        self.client = MongoClient(mongodb_uri)
        self.db = self.client['leagueData']
        self.summoner_collection = self.db['summoner_data']
        self.puuid_collection = self.db['puuid_data']

    def convert_summoner_ids_to_puuids(self):
        summoners = self.summoner_collection.find()
        for summoner in summoners:
            puuid = self.api.get_puuid(summoner['summonerId'])
            if puuid:
                self.puuid_collection.insert_one({
                    'summonerId': summoner['summonerId'],
                    'puuid': puuid,
                    'tier': summoner['tier'],
                    'division': summoner['division']
                })
                print(f"Stored PUUID and rank for {summoner['summonerId']}: {puuid}, {summoner['tier']} {summoner['division']}")


class MatchRetriever:
    def __init__(self, api_key, mongodb_uri):
        self.api = RiotAPI(api_key)
        self.client = MongoClient(mongodb_uri)
        self.db = self.client['leagueData']
        self.puuid_collection = self.db['puuid_data']
        self.match_collection = self.db['match_data']

    def retrieve_matches(self):
        puuids = self.puuid_collection.find()
        for puuid_data in puuids:
            matches = self.api.get_match_history(puuid_data['puuid'])
            if matches:
                self.match_collection.insert_many([{'matchId': match} for match in matches])



# Execution

In [ ]:
if __name__ == "__main__":
    api_key = x
    mongodb_uri = 'mongodb://localhost:27017/'

    # Define all tiers and divisions you want to gather data for
    tiers = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "DIAMOND"]
    divisions = ["IV", "III", "II", "I"]

    # Create an instance of the SummonerDataCollector
    summoner_collector = SummonerDataCollector(api_key, mongodb_uri)

    # Loop through each tier and division and collect data
    for tier in tiers:
        for division in divisions:
            print(f"Gathering data for {tier} {division}...")
            summoner_collector.gather_summoner_data(tier, division)
            print(f"Data collection complete for {tier} {division}.")

    # Assuming you want to proceed immediately to the next steps after data collection:
    # Step 2: Convert Summoner IDs to PUUIDs
    puuid_converter = PUUIDConverter(api_key, mongodb_uri)
    puuid_converter.convert_summoner_ids_to_puuids()

    # Step 3: Retrieve Matches using PUUIDs
    match_retriever = MatchRetriever(api_key, mongodb_uri)
    match_retriever.retrieve_matches()
